In [ ]:
from raiwidgets import FairnessDashboard
from sklearn.model_selection import train_test_split

from sklearn import svm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
import pandas as pd

In [ ]:
from sklearn.datasets import fetch_openml
data = fetch_openml(data_id=1590, as_frame=True)
X_raw = data.data
Y = (data.target == '>50K') * 1
X_raw

In [ ]:
A = X_raw[['sex','race']]
X = X_raw.drop(labels=['sex', 'race'],axis = 1)
X = pd.get_dummies(X)

sc = StandardScaler()
X_scaled = sc.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

le = LabelEncoder()
Y = le.fit_transform(Y)


X_train,\
    X_test,\
    Y_train,\
    Y_test,\
    A_train,\
    A_test = train_test_split(X_scaled,
                              Y,
                              A,
                              test_size=0.2,
                              random_state=0,
                              stratify=Y)


X_train = X_train.reset_index(drop=True)
A_train = A_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
A_test = A_test.reset_index(drop=True)


In [ ]:
lr_predictor = LogisticRegression(
    solver='liblinear', fit_intercept=True)

lr_predictor.fit(X_train, Y_train)

#svm_predictor = svm.SVC()

#svm_predictor.fit(X_train, Y_train)

In [ ]:
lr_predictor.predict(X_test)

In [ ]:
FairnessDashboard(sensitive_features=A_test,
                  y_true=Y_test,
                  y_pred=lr_predictor.predict(X_test)
)

In [ ]:
svm_predictor = svm.SVC()

svm_predictor.fit(X_train, Y_train)

In [ ]:
FairnessDashboard(sensitive_features=A_test,
                  y_true=Y_test,
                  y_pred={
                      "SVM":svm_predictor.predict(X_test)
                  })

In [ ]:
FairnessDashboard(sensitive_features=A_test,
                  y_true=Y_test,
                  y_pred={
                      "Log. Reg.":lr_predictor.predict(X_test),
                      "SVM":svm_predictor.predict(X_test)
                  })

In [1]:
# Copyright (c) Microsoft Corporation
# Licensed under the MIT License.

from raiwidgets import ModelAnalysisDashboard
from responsibleai import ModelAnalysis
import sklearn
import shap
import pandas as pd

x, y = shap.datasets.adult()
y = [1 if r else 0 for r in y]

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(
    x, y, test_size=0.2, random_state=7)

knn = sklearn.neighbors.KNeighborsClassifier()
knn.fit(x_train, y_train)


train = pd.merge(x, pd.DataFrame(
    y, columns=["income"]), left_index=True, right_index=True)
test = pd.merge(x_test, pd.DataFrame(y_test, columns=[
                "income"]), left_index=True, right_index=True)


print(train)
print(test)


ma = ModelAnalysis(knn, train, test, "income", "classification",
                   categorical_features=[])
ma.explainer.add()
ma.counterfactual.add(['Age', 'Workclass', 'Education-Num', 'Marital Status',
                       'Occupation', 'Relationship', 'Race', 'Sex',
                       'Capital Gain', 'Capital Loss',
                       'Hours per week', 'Country'], 10,
                      desired_class="opposite")
ma.error_analysis.add()
ma.causal.add()
ma.compute()


C:\Users\xuke\Miniconda3\envs\dev\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
cuML is required to use GPU explainers. Check https://rapids.ai/start.html         for more information on how to install it.
        Age  Workclass  Education-Num  Marital Status  Occupation  \
0      39.0          7           13.0               4           1   
1      50.0          6           13.0               2           4   
2      38.0          4            9.0               0           6   
3      53.0          4            7.0               2           6   
4      28.0          4           13.0               2          10   
...     ...        ...            ...             ...         ...   
32556  27.0          4           12.0               2          13   
32557  40.0          4            9.0               2           7   
32558  58.0        

In [5]:
causal = ma.causal.get()[0]


In [6]:
print (causal)

{'global_causal_effects':                    point        stderr      zstat        p_value  ci_lower  \
feature                                                                      
Age             0.002986  1.857130e-04  16.078655   3.601121e-58  0.002622   
Workclass      -0.009340  1.726048e-03  -5.411289   6.257275e-08 -0.012723   
Education-Num   0.024684  8.755976e-04  28.191510  7.431312e-175  0.022968   
Marital Status -0.002504  2.050662e-03  -1.220969   2.220978e-01 -0.006523   
Occupation      0.000432  4.687621e-04   0.922108   3.564724e-01 -0.000487   
Relationship   -0.003922  2.201144e-03  -1.781790   7.478352e-02 -0.008236   
Race            0.005440  2.487220e-03   2.187351   2.871690e-02  0.000566   
Sex             0.038619  6.698869e-03   5.764979   8.166805e-09  0.025489   
Capital Gain    0.000013  7.960074e-07  16.116402   1.956767e-58  0.000011   
Capital Loss    0.000072  6.084807e-06  11.864600   1.807634e-32  0.000060   
Hours per week  0.002150  1.595970e-04

In [13]:
import json
json.loads(causal["global_causal_effects"].reset_index().to_json(orient = "records"))

[{'feature': 'Age',
  'point': 0.0029860149,
  'stderr': 0.000185713,
  'zstat': 16.0786550906,
  'p_value': 3.601121208e-58,
  'ci_lower': 0.0026220242,
  'ci_upper': 0.0033500057},
 {'feature': 'Workclass',
  'point': -0.009340143,
  'stderr': 0.0017260478,
  'zstat': -5.4112888315,
  'p_value': 6.26e-08,
  'ci_lower': -0.0127231345,
  'ci_upper': -0.0059571516},
 {'feature': 'Education-Num',
  'point': 0.0246844181,
  'stderr': 0.0008755976,
  'zstat': 28.1915101505,
  'p_value': 7.431312255e-175,
  'ci_lower': 0.0229682784,
  'ci_upper': 0.0264005579},
 {'feature': 'Marital Status',
  'point': -0.002503795,
  'stderr': 0.0020506625,
  'zstat': -1.220968862,
  'p_value': 0.2220978096,
  'ci_lower': -0.0065230196,
  'ci_upper': 0.0015154296},
 {'feature': 'Occupation',
  'point': 0.0004322491,
  'stderr': 0.0004687621,
  'zstat': 0.9221077107,
  'p_value': 0.3564723954,
  'ci_lower': -0.0004865076,
  'ci_upper': 0.0013510059},
 {'feature': 'Relationship',
  'point': -0.0039219761,
  

In [ ]:
json_data = causal.global_causal_effect.reset_index().to_json(orient = "records")
import json

json_object = json.loads(json_data)

with open('d:\causal.json', 'w') as f:
    json.dump(json_object, f, indent=2)


In [ ]:
 local = causal.local_causal_effect(x_test.head(5))

In [ ]:
json_data = local.reset_index().to_json(orient = "records")
import json

json_object = json.loads(json_data)

with open('d:\causal.json', 'w') as f:
    json.dump(json_object, f, indent=2)


In [ ]:
import json

In [ ]:
grouped = local.groupby("sample").apply(lambda x: x.reset_index().to_json(orient='records'))

In [ ]:
grouped.sort_by

In [ ]:
iris = sklearn.datasets.load_iris()

In [ ]:
x = pd.DataFrame(
    iris.data, columns=iris.feature_names)
y = pd.DataFrame(
    iris.target, columns=["class"])

In [ ]:

from raiwidgets import ModelAnalysisDashboard
from responsibleai import ModelAnalysis
import sklearn
import shap
import pandas as pd

x = pd.DataFrame(
    iris.data, columns=iris.feature_names)
y = pd.DataFrame(
    iris.target, columns=["class"])

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(
    x, y, test_size=0.2, random_state=7)

knn = sklearn.neighbors.KNeighborsClassifier()
knn.fit(x_train, y_train)


train = pd.merge(x, y, left_index=True, right_index=True)
test = pd.merge(x_test, y_test, left_index=True, right_index=True)


In [ ]:



ma = ModelAnalysis(knn, train, test, "class", "classification",
                   categorical_features=[])
ma.explainer.add()
ma.counterfactual.add(iris.feature_names, 10,
                      desired_class=1)
ma.error_analysis.add()
ma.causal.add()
ma.compute()